In [ ]:
Django signals are a way to allow decoupled applications to get notified when certain actions occur 
elsewhere in the application. They allow certain senders to notify a set of receivers when some 
action has taken place. Here's a deeper dive into Django signals with a live project example and 
scenarios where they can be useful.

Overview of Django Signals
Django provides several built-in signals that can be used, including:

1.pre_save and post_save: Sent before or after a model’s save() method is called.
2.pre_delete and post_delete: Sent before or after a model’s delete() method is called.
3.m2m_changed: Sent when a ManyToManyField is changed.

Creating and Using Custom Signals
In addition to the built-in signals, you can also create custom signals.

Example: Sending a Welcome Email After User Registration
In this example, we will use the post_save signal to send a welcome email to a user after they have 
registered.

Step-by-Step Implementation
Step 1: Define the Signal Receiver
Create a signal receiver function that will send an email.

In [ ]:
# myapp/signals.py

from django.contrib.auth.models import User
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.core.mail import send_mail
from django.conf import settings

@receiver(post_save, sender=User)
def send_welcome_email(sender, instance, created, **kwargs):
    if created:
        subject = 'Welcome to MySite'
        message = f'Hi {instance.username}, thank you for registering at MySite.'
        from_email = settings.DEFAULT_FROM_EMAIL
        recipient_list = [instance.email]
        send_mail(subject, message, from_email, recipient_list)


In [ ]:
Step 2: Connect the Signal
Ensure that the signal is connected when the application starts. You can do this in the apps.py of 
your app.

# myapp/apps.py

from django.apps import AppConfig

class MyAppConfig(AppConfig):
    name = 'myapp'

    def ready(self):
        import myapp.signals  # noqa


In [ ]:
# settings.py

INSTALLED_APPS = [
    ...
    'myapp.apps.MyAppConfig',  # Ensure you use the custom AppConfig
    ...
]

# Email settings (update these with your own email configuration)
EMAIL_BACKEND = 'django.core.mail.backends.smtp.EmailBackend'
DEFAULT_FROM_EMAIL = 'webmaster@mysite.com'
EMAIL_HOST = 'smtp.example.com'
EMAIL_PORT = 587
EMAIL_USE_TLS = True
EMAIL_HOST_USER = 'your-email@example.com'
EMAIL_HOST_PASSWORD = 'your-email-password'


In [ ]:
Step 3: Create and Save a User
Whenever a new user is created, the send_welcome_email signal receiver will be triggered.
# In a view or any other place where user registration happens
from django.contrib.auth.models import User

# Create a new user
new_user = User.objects.create_user(username='newuser', email='newuser@example.com', password='password')



In [ ]:
Explanation
Signal Receiver: The send_welcome_email function is a signal receiver that listens for the post_save
    signal from the User model. When a new user is created (created=True), it sends a welcome email.
    
Connecting the Signal: The receiver is connected to the signal in apps.py, ensuring that the 
    connection happens when the app is ready.
Triggering the Signal: When a new User instance is saved, the post_save signal is sent, triggering
    the send_welcome_email function.
Real-World Use Cases for Django Signals

User Activity Logging:

Scenario: You want to log every time a user logs in or logs out.
Signal: user_logged_in, user_logged_out.
Order Processing:

Scenario: When an order is created, you want to update the inventory and send a confirmation email.
Signal: post_save on the Order model.
Profile Creation:

Scenario: Automatically create a user profile when a new user registers.
Signal: post_save on the User model.
Notification System:

Scenario: Notify users when someone comments on their post.
Signal: Custom signal emitted when a new comment is created.
